In [40]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.optimize import minimize


C is the list (of lists) of calibration tuples $(p(t_i), \tilde{q}(t_j), t_l)$, where $t_l$ is the joint measurement time $t_l = (t_i + t_j) / 2$. $p(t_i)$ is the measured value. $\tilde{q}(t_j)$ is the calibration provided by another sensor.
I've made C a list of lists, one for each sensor.

We adjust $\gamma_1(T_m)$ to minimise the least squares error between $c(\gamma(T_m),p(t_i))$ and $\tilde{q}(t_j)$.

In [48]:
X = []
Y = []
X.append([ 0,0,1]); Y.append([15,15])
X.append([ 1,0,1]); Y.append([12,12])
X.append([ 2,0,1]); Y.append([10,10])
X.append([20,2,3]); Y.append([20,10])
X.append([33,1,2]); Y.append([10,20])
X.append([34,1,2]); Y.append([20,40])
X.append([35,1,2]); Y.append([30,60])
X.append([36,0,3]); Y.append([10,10])
X = np.array(X)
Y = np.array(Y)
gamma = [[1,0],[0,0],[0,0],[0,0]]
lasttime = 0
Tnow = 36 #time now

In [49]:
def getageofparents(cs,Tnow):
    if len(cs)==0: return 10000 #lots = i.e. not been calibrated
    cparents = np.array(cs)[:,3]
    cparents = cparents[cparents<1000]
    if len(cparents)==0:
        return 10000 #lots
    else:
        return np.mean(cparents)+np.min(Tnow-np.array(C[x[1]])[:,2])

In [50]:
C = [[],[],[],[]]

In [51]:


def w(deltat):
    return 1/(1+deltat)
def cal(params,pollution):
    return params[0]*pollution+params[1]
def geterror(cs,g):
    s = 0
    for c in cs:
        s+=w(Tnow-c[2])*(cal(g,c[0])-c[1])**2
    return s

np.set_printoptions(precision=3,suppress=True)
for x,y in zip(X,Y):
    #Quoting the paper:
    #Hence, we define the age of a calibration as the sum of (i) the time passed since a
    #sensor’s most recent calibration T m and (ii) the average calibration age of the sensor’s
    #calibration parents
    
    #np.mean(np.array(C[x[1]])[:,3]) #average age of sensors calibration parents
    #np.min(Tnow-np.array(C[x[1]])[:,2]) #time passed since most recent calibration
    
    print("Before")
    print("x[1]=",x[1],"C[x[1]]=",C[x[1]])
    print("x[2]=",x[2],"C[x[2]]=",C[x[2]])
    t1 = getageofparents(C[x[1]],Tnow)
    t2 = getageofparents(C[x[2]],Tnow)
    
    if (x[1]==0) or (x[2]==0):
        t1 = 0
        t2 = 0
        C[x[2]]=[]
        C[x[1]]=[]
    tl = x[0]
    if (x[1]==0):
        pol = y[0]
    else:
        if (x[2]==0):
            pol = y[1]
        else:
            pol = (t1*cal(gamma[x[2]],y[1])+t2*cal(gamma[x[1]],y[0]))/(t1+t2)
    
    C[x[1]].append([y[0],pol,tl,min(t1,t2)])
    C[x[2]].append([y[1],pol,tl,min(t1,t2)])
    print("pollution=",pol,"t1=",t1,"gamma[x1]=",gamma[x[1]],"-->",cal(gamma[x[1]],y[0]),"|")
    print("              ","t2=",t2,"gamma[x2]=",gamma[x[2]],"-->",cal(gamma[x[2]],y[1]),"|")
    print("-----")
    for i,(cs,g) in enumerate(zip(C,gamma)):
        res = minimize(lambda newg: geterror(cs,newg), g)
        gamma[i] = res.x
    print("-----")
    print(gamma)
    print("After")
    print("x[1]=",x[1],"C[x[1]]=",C[x[1]])
    print("x[2]=",x[2],"C[x[2]]=",C[x[2]])
    print("========================")
    

Before
x[1]= 0 C[x[1]]= []
x[2]= 1 C[x[2]]= []
pollution= 15 t1= 0 gamma[x1]= [1, 0] --> 15 |
               t2= 0 gamma[x2]= [0, 0] --> 0 |
-----
-----
[array([1., 0.]), array([0.996, 0.066]), array([0., 0.]), array([0., 0.])]
After
x[1]= 0 C[x[1]]= [[15, 15, 0, 0]]
x[2]= 1 C[x[2]]= [[15, 15, 0, 0]]
Before
x[1]= 0 C[x[1]]= [[15, 15, 0, 0]]
x[2]= 1 C[x[2]]= [[15, 15, 0, 0]]
pollution= 12 t1= 0 gamma[x1]= [1. 0.] --> 12.0 |
               t2= 0 gamma[x2]= [0.996 0.066] --> 12.013274248182727 |
-----
-----
[array([1., 0.]), array([0.994, 0.066]), array([0., 0.]), array([0., 0.])]
After
x[1]= 0 C[x[1]]= [[12, 12, 1, 0]]
x[2]= 1 C[x[2]]= [[12, 12, 1, 0]]
Before
x[1]= 0 C[x[1]]= [[12, 12, 1, 0]]
x[2]= 1 C[x[2]]= [[12, 12, 1, 0]]
pollution= 10 t1= 0 gamma[x1]= [1. 0.] --> 10.0 |
               t2= 0 gamma[x2]= [0.994 0.066] --> 10.011046664309275 |
-----
-----
[array([1., 0.]), array([0.993, 0.066]), array([0., 0.]), array([0., 0.])]
After
x[1]= 0 C[x[1]]= [[10, 10, 2, 0]]
x[2]= 1 C[x[2]]= [

In [52]:
#newX = X[-1::-1,:]
#newX[:,0]=-newX[:,0]+36
#X=newX


In [53]:
for c in C: print(c,'\n')

[[10, 10, 36, 0]] 

[[10, 10, 2, 0], [10, 9.966115197949653, 33, 34.0], [20, 18.554779585971612, 34, 20.0], [30, 27.916204245157193, 35, 20.0]] 

[[20, 0.0, 20, 10000], [20, 9.966115197949653, 33, 34.0], [40, 18.554779585971612, 34, 20.0], [60, 27.916204245157193, 35, 20.0]] 

[[10, 10, 36, 0]] 



In [54]:
gamma

[array([1., 0.]),
 array([0.901, 0.803]),
 array([ 0.494, -1.605]),
 array([0.99 , 0.099])]

why is the the age of a calibration 
> "the sum of (i) the time passed since a sensor’s most recent calibration T m and (ii) the average calibration age of the sensor’s calibration parents"

surely if one of the parents is a reference then... also this isn't very clear. What are its parents? The two sensors?

In [74]:

#C = [[(10,14.0,33),(20,24,34),(30,34,35),(15,15,0)]]


for cs,g in zip(C,gamma):
    geterror(cs,g)

In [37]:
from scipy.optimize import minimize
res = minimize(lambda newg: geterror(cs,newg), g)
newg = res.x
newg

array([1.01091164, 3.66116439])